In [ ]:
from azmeta.access import resource_graph, reservations, list_subscription_ids, resource_id
from azmeta.notebook.context import AzureCliContext
import pandas as pd
import csv

# Parameters

In [ ]:
arm_location = "westeurope"

## Load RI Family Data

In [ ]:
flex_families = pd.read_csv("ISFRatio.csv")
flex_families = flex_families.assign(ArmSkuName=flex_families.ArmSkuName.str.lower()).set_index('ArmSkuName')

# Pull Resources

In [ ]:
context = AzureCliContext()
all_subscription_ids = list_subscription_ids(context.subscriptions)
vms = resource_graph.query_dataframe(all_subscription_ids, f"""
Resources 
| where type == 'microsoft.compute/virtualmachines' and location == '{arm_location}'
| join kind=leftouter (ResourceContainers | where type == 'microsoft.resources/subscriptions' | project subscriptionName=name, subscriptionId) on subscriptionId
| extend vmSize = tolower(properties.hardwareProfile.vmSize)
| summarize instanceCount=count() by subscriptionId, subscriptionName, vmSize
| order by subscriptionName asc
""")

In [ ]:
flex_vms = vms.join(flex_families, on='vmSize')
flex_vms = flex_vms.assign(InstanceSizeFlexibilityGroup=flex_vms.InstanceSizeFlexibilityGroup.fillna('NO_FAMILY')) \
                   .assign(Ratio=flex_vms.Ratio.fillna(1.0))
flex_vms = flex_vms.assign(normalizedUnits=flex_vms.instanceCount * flex_vms.Ratio)
sub_names = flex_vms.filter(['subscriptionId', 'subscriptionName']).drop_duplicates().set_index('subscriptionId')
flex_vms.style.hide_index()

In [ ]:
flex_vms_by_sub_group = flex_vms.drop(columns=['instanceCount', 'Ratio', 'vmSize']).groupby(['subscriptionName', 'InstanceSizeFlexibilityGroup'], as_index=False).sum() 
flex_vms_by_sub_group.style.hide_index()

# Pull Reservations

In [ ]:
raw_reservations = reservations.reservations_dataframe()

In [ ]:
raw_reservations = raw_reservations[raw_reservations.provisioning_state == "Succeeded"]
if len(raw_reservations[(raw_reservations.applied_scope_type != 'Single') | (raw_reservations.applied_scopes.map(lambda x: len(x)) != 1)]):
    raise 'Unsupported'
raw_reservations = raw_reservations \
    .assign(subscriptionId=raw_reservations.applied_scopes.map(lambda x: resource_id.subscription_id(x[0]))) \
    .assign(sku=raw_reservations.sku.str.lower()) \
    .drop(columns='name') \
    .rename(columns={'sku': 'vmSize', 'display_name': 'name'}) \
    .filter(['vmSize', 'quantity', 'subscriptionId', 'name'])

In [ ]:
flex_reservations = raw_reservations.join(flex_families, on='vmSize')
flex_reservations = flex_reservations.assign(normalizedUnits=flex_reservations.quantity * flex_reservations.Ratio)
flex_reservations[flex_reservations.subscriptionId.isin(sub_names.index)].sort_values('name').style.hide_index()

In [ ]:
flex_reservations_by_sub_group = flex_reservations.drop(columns=['quantity', 'Ratio', 'vmSize']).groupby(['subscriptionId', 'InstanceSizeFlexibilityGroup'], as_index=False).agg({'normalizedUnits': 'sum', 'name': lambda s: '<br/>'.join(s)})
flex_reservations_by_sub_group = flex_reservations_by_sub_group.join(sub_names, on='subscriptionId', how='inner').drop(columns='subscriptionId')
display(flex_reservations_by_sub_group[['subscriptionName', 'InstanceSizeFlexibilityGroup', 'normalizedUnits', 'name']].style)
flex_reservations_by_sub_group = flex_reservations_by_sub_group.drop(columns='name')

In [ ]:
flex_reservations_by_sub_group.style

# Report Out

In [ ]:
left=flex_vms_by_sub_group.set_index(['subscriptionName', 'InstanceSizeFlexibilityGroup'])
right=flex_reservations_by_sub_group.set_index(['subscriptionName', 'InstanceSizeFlexibilityGroup'])
report = left.join(right, rsuffix='Reserved', how='outer')
report = report.assign(reservationShortage=report.normalizedUnits.fillna(0.0) - report.normalizedUnitsReserved.fillna(0.0))
report.style.background_gradient(subset='reservationShortage').format('{:g}', na_rep='0')

In [ ]:
report.swaplevel(0,1).sort_index().style.background_gradient(subset='reservationShortage').format('{:g}', na_rep='0')